In [116]:
import pandas as pd
import config as c
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

In [159]:
data = pd.read_csv('data/runs/results.csv', index_col='run')

In [118]:
#data = data[data['status']!='error']
#data = data[data['r_2_mean'] > 0]

In [119]:
data['time'] = pd.to_timedelta(data['time'])

In [120]:
data['seconds'] = data['time'].apply(lambda x:x.seconds)

In [121]:
data.dtypes

operation             object
started               object
time         timedelta64[ns]
status                object
label                 object
ccle                  object
ctrp                  object
cutoff               float64
drugs                float64
feda                  object
fs                    object
gdsc                  object
metric                object
model                 object
n                    float64
test                 float64
threshold            float64
step                 float64
r_2_mean             float64
r_2_std              float64
seconds                int64
dtype: object

In [160]:
data['datasets'] = data['gdsc']+data['ctrp']+data['ccle']

In [164]:
data[(data['datasets']>1)&(data['feda'] == True)].describe()

,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std
count,185.0,185.000000,182.000000,0.0,185.000000,180.0,180.000000,180.000000
mean,4.0,20.881081,0.102455,NaN,0.010703,0.0,-0.022006,0.031652
std,0.0,5.489805,0.065033,NaN,0.007226,0.0,0.032068,0.075356
min,4.0,3.000000,0.000000,NaN,0.010000,0.0,-0.192936,0.001156
25%,4.0,20.000000,0.048302,NaN,0.010000,0.0,-0.021584,0.006621
50%,4.0,20.000000,0.099347,NaN,0.010000,0.0,-0.007634,0.011621
75%,4.0,20.000000,0.153111,NaN,0.010000,0.0,-0.004286,0.023433
max,4.0,50.000000,0.245415,NaN,0.100000,0.0,-0.001289,0.721564


In [124]:
data.describe()

,time,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds
count,590,590.0,590.000000,586.000000,0.0,590.000000,590.0,590.000000,590.000000,590.000000
mean,0 days 00:33:04.457627,4.0,21.394915,0.104147,NaN,0.010153,0.0,0.541444,0.175188,1984.457627
std,0 days 00:46:24.540358,0.0,6.428526,0.060223,NaN,0.003705,0.0,0.382944,0.252005,2784.540359
min,0 days 00:00:35,4.0,3.000000,0.000266,NaN,0.010000,0.0,0.007302,0.000018,35.000000
25%,0 days 00:08:25.250000,4.0,20.000000,0.056068,NaN,0.010000,0.0,0.168165,0.017592,505.250000
50%,0 days 00:20:14,4.0,20.000000,0.099197,NaN,0.010000,0.0,0.383953,0.128729,1214.000000
75%,0 days 00:39:17.750000,4.0,20.000000,0.157270,NaN,0.010000,0.0,0.988549,0.182340,2357.750000
max,0 days 05:20:25,4.0,50.000000,0.244458,NaN,0.100000,0.0,0.999978,1.711669,19225.000000


## FEDA t-test

In [136]:
data[data['fs']=='Lasso'][['model', 'feda','r_2_mean']]

,model,feda,r_2_mean
run,,,
43650dda,DecisionTreeRegressor,False,0.999966
ffb505bc,DecisionTreeRegressor,False,0.999978
72de3f17,DecisionTreeRegressor,True,0.999360
b1b3861e,DecisionTreeRegressor,False,0.999041
d333ba13,DecisionTreeRegressor,False,0.997707
74d853e6,DecisionTreeRegressor,False,0.999439
e906f40a,DecisionTreeRegressor,False,0.999811
2acb793f,DecisionTreeRegressor,True,0.998952
225b2685,DecisionTreeRegressor,False,0.999966


In [137]:
from scipy import stats
stats.ttest_ind(data[data['metric']=='AUC_IC50']['r_2_mean'].values, data[data['metric']!='AUC_IC50']['r_2_mean'].values)

Ttest_indResult(statistic=0.40994050112150177, pvalue=0.6819989037587484)

In [128]:
stats.ttest_ind(data[data['feda']==True]['r_2_mean'].values, data[data['feda']==False]['r_2_mean'].values, equal_var=False)

Ttest_indResult(statistic=-2.0976882970920734, pvalue=0.036946997374768226)

## Model composition

In [96]:
models = data.groupby('model').mean()
models['Number of runs'] = [data[data['model']==i].shape[0] for i in models.index]
models

,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds,Number of runs
model,,,,,,,,,,
DecisionTreeRegressor,4.0,20.000000,0.095306,NaN,0.01000,0.0,0.763392,0.027828,1862.804511,133
ElasticNet,4.0,20.000000,0.100454,NaN,0.01000,0.0,0.154809,0.122977,1476.758950,419
KNeighborsRegressor,4.0,20.000000,0.082703,NaN,0.01000,0.0,0.246350,0.081791,4744.200000,20
RandomForestRegressor,4.0,19.900585,0.106795,NaN,0.01000,0.0,0.806358,0.019549,3646.461988,171
SVR,4.0,30.324000,0.122857,NaN,0.01124,0.0,0.151521,1.087878,1475.313008,250


## Individual drug analysis

In [35]:
svr = data[(data['model'] == 'SVR') & (data['r_2_mean']>0)]
svr[svr['r_2_mean']> 0.6][['feda', 'n', 'fs','r_2_mean']].sort_values('r_2_mean', ascending=False)

,feda,n,fs,r_2_mean
run,,,,
c84c194d,False,0.010000,ElasticNet,0.934915
8071f94a,False,0.094795,DecisionTreeRegressor,0.922619
400d39b7,False,0.098271,DecisionTreeRegressor,0.922580
d2703cd1,False,0.147682,mutual_info_regression,0.919401
dfde6ba7,False,0.072084,mutual_info_regression,0.917935
c889fa24,False,0.087590,f_regression,0.916772
b1cea166,False,0.003354,mutual_info_regression,0.901983
e32273a7,False,0.144099,ElasticNet,0.897122
9cabe208,False,0.057282,RandomForestRegressor,0.887763


In [36]:
en = data[(data['model'] == 'ElasticNet') & (data['r_2_mean']>0)]
en[['feda', 'n', 'fs','r_2_mean']].sort_values('r_2_mean', ascending=False)

,feda,n,fs,r_2_mean
run,,,,
150587db,False,0.100081,mutual_info_regression,0.541867
140511c0,False,0.006392,DecisionTreeRegressor,0.502989
61f79f6a,False,0.146809,DecisionTreeRegressor,0.499752
a0fd39fd,False,0.161861,DecisionTreeRegressor,0.479983
d122f81c,True,0.002680,DecisionTreeRegressor,0.445192
...,...,...,...,...
0485e171,False,0.066889,f_regression,0.032737
34031797,False,0.136819,f_regression,0.032026
ae6aad99,True,0.083612,f_regression,0.030006


In [138]:
dt = data[(data['model'] == 'DecisionTreeRegressor') & (data['r_2_mean']>0)]
dt[['feda', 'n', 'fs','r_2_mean']].sort_values('r_2_mean', ascending=False)

,feda,n,fs,r_2_mean
run,,,,
ffb505bc,False,0.029654,Lasso,0.999978
8ee1461e,False,0.000266,Lasso,0.999976
43650dda,False,0.073360,Lasso,0.999966
225b2685,False,0.001797,Lasso,0.999966
4976193b,False,0.176037,Lasso,0.999955
...,...,...,...,...
e5abd2b2,True,0.035179,mutual_info_regression,0.973453
8185a6f9,True,0.165552,RandomForestRegressor,0.960727
fb547ef6,True,0.015456,f_regression,0.958145


In [139]:
def append(data, run):
    result = pd.DataFrame()
    for a in run:
        f = subprocess.run("guild select " +a, shell=True, capture_output=True)
        d = pd.read_csv('venv/.guild/runs/'+f.stdout.decode('utf-8')[:-1]+'/scores.csv')
        d['run'] = [a for i in range(0, d.shape[0])]
        d = d.set_index('run')
        result = pd.concat([result, d])
    return result

In [143]:
k = append(dt, dt.index)

In [175]:
data[(data['model']=='DecisionTreeRegressor') & (data['datasets']>1)][['metric','fs','ctrp','ccle','gdsc', 'datasets', 'n', 'r_2_mean']]

,metric,fs,ctrp,ccle,gdsc,datasets,n,r_2_mean
run,,,,,,,,
43650dda,AUC_EC50,Lasso,True,True,True,3,0.073360,0.999966
ffb505bc,AUC_EC50,Lasso,True,True,True,3,0.029654,0.999978
b1b3861e,AUC_IC50,Lasso,True,False,True,2,0.015029,0.999041
a0f28d2e,AUC_IC50,Lasso,False,True,True,2,0.190593,-0.039144
74d853e6,AUC_IC50,Lasso,True,True,False,2,0.075327,0.999439
...,...,...,...,...,...,...,...,...
ed20f339,AUC_EC50,ElasticNet,True,True,False,2,0.014686,0.980698
dde14bbd,AUC_EC50,Lasso,True,True,False,2,0.106676,0.999529
76f95344,AUC_EC50,RandomForestRegressor,False,True,True,2,0.199108,0.992276


In [153]:
k.loc['72de3f17']

,Unnamed: 0,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,criterion,max_depth,max_features,max_leaf_nodes,min_impurity_decrease,min_impurity_split,min_samples_leaf,min_samples_split,min_weight_fraction_leaf,presort,random_state,splitter
run,,,,,,,,,,,,,,,,,
72de3f17,BRD-A71883111,0.999502,0.000836,0.000009,0.000160,mae,12,log2,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,BRD-K66453893,0.999937,0.001281,0.000004,0.000885,mae,38,auto,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,CAY10576,0.998612,0.000764,0.000011,0.000036,mae,16,sqrt,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,CHM-1,0.999938,0.001428,0.000004,0.001076,mae,34,sqrt,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,Compound 23 citrate,0.999724,0.001584,0.000010,0.000530,mae,36,log2,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,Dinaciclib,0.999811,0.001738,0.000009,0.000587,mae,16,log2,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,ELCPK,0.999767,0.003032,0.000016,0.002300,mae,26,sqrt,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,LRRK2-IN-1,0.999868,0.002021,0.000009,0.001231,mae,22,log2,NaN,0.0,NaN,1,2,0.0,False,NaN,best
72de3f17,ML006,0.999847,0.001124,0.000006,0.000293,mae,32,auto,NaN,0.0,NaN,1,2,0.0,False,NaN,best


In [84]:
k.groupby('normalize').mean()

,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,alpha,copy_X,fit_intercept,l1_ratio,max_iter,positive,precompute,random_state,tol,warm_start
normalize,,,,,,,,,,,,,,
False,0.300160,0.137149,0.032678,0.114441,0.787859,1.0,1.000000,0.012693,1000,0.0,0.0,NaN,0.0001,0.0
True,0.218647,0.124107,0.028921,0.107119,0.490850,1.0,0.496732,0.043301,1000,0.0,0.0,NaN,0.0001,0.0


In [34]:
plt.scatter(k['l1_ratio'], k['r2_score'])

NameError: name 'k' is not defined